###### <span style="color:#cc7a00">Critical Zone Observatory - Modeling Institute 2016</span>

<img style="float: right;" src="http://hydrocomplexity.net/images/dharaD.png" width="70px" hspace=5 /> <br/>

# Tutorial: Introduction to Dhara<sup>$\dagger$</sup> model
*<sup>$\dagger$</sup>Sanskrit, noun: earth, flow <br>*

## 1. What is Dhara?

[Dhara](https://hydrocomplexity.github.io/Dhara/) is an open-source, high-performance computing framework for modeling complex processes in the __Critical Zone__ (CZ). The model is designed to be modular in structure with the aim to create uniform, efficient, and interdisciplinary tools to facilitate and leverage process modeling. It also provides flexibility to maintain, collaborate, and co-develop new components by the scientific community. Currently, Dhara consists of two main components (See Figure 1):
* A <span style="color:#cc7a00">**multi-layer canopy**</span> model to simulate within-canopy processes in the aboveground,
* An <span style="color:#cc7a00">**integrated flow**</span> model which includes overland and subsurface modules to simulate vertical and horizontal fluxes on the surface and in the belowground system.

<img src="http://hydrocomplexity.net/images/dhara/Dhara_main.png" alt="" width="600">

</br>
<center>*__Figure 1__. Schematic of Dhara model on hybrid CPU-GPU parallel computing architecture.*</center>

Dhara is primarily developed to utilize *high-resolution topographic* data (particularly *lidar*) for simulations of complex (i.e. ecohydrologic and biogeochemical) processes dominated by topographic *controls* and *variability* across scales in the CZ. 

## 2. Organization

Dhara modeling framework uses hybrid parallel computing which combines the advantages of [Message Passing Interface (MPI)](https://www.open-mpi.org/) and [GPU parallel computing](http://www.nvidia.com/object/what-is-gpu-computing.html) for optimal performance on large-scale simulations. 
The model is written using [CUDA C/C++](https://blogs.nvidia.com/blog/2012/09/10/what-is-cuda-2/) parallel computing platform and programming model that are based on the industry-standard C/C++.
With this approach, users can accelerate their code by moving the computationally intensive, data parallelism portion to a GPU and still utilize MPI for portions that are best suited to task parallelism for optimal performances.

### 2.1 Structure
The structure of Dhara source code is arranged in 4 different levels as shown in Figure 2.
* __Level 1__ - main program entry point (`main.cc`): Program starts and controls MPI environments and numerical modules;
* __Level 2__ - numerical (`numerical.cc`) and MPI (`mpienv.cc`) modules: While `mpienv.cc` provides functions for MPI , `numerical.cc` includes functions and links to level 3 files for inputs/outputs (I/O) and numerical simulations.
* __Level 3__ - processing modules: includes files that provide tools to process I/O (`io.cc`) in `Dhara`, parse configurations (`parser.cc`), and execute host (`host.cc`) and device (`device.cu`) functions.
* __Level 4__ - model solvers: includes source files for numerical solvers for different modules used in Dhara. Most of the level 4 files are linked with the `device.cu` where all the models communicate to each other.

<img src="http://hydrocomplexity.net/images/dhara/dhara_codestructure.png?nocache=<?php echo time(); ?>" alt="" width="600">

</br>
<center>*__Figure 2__. Organization of source files in Dhara. User-defined classes for Dhara model is shown on the right*</center>

User-defined classes are defined in header files. Dhara source code is built and compiled using `Makefile`:
* All `*.cc` files are compiled with `GNU gcc` compiler with a link to CUDA (`lcudart`) library.
* All `*.cu` files are compiled with `nvcc` compiler with a link to MPI. 
* NetCDF library is linked to Dhara for I/O using array-oriented scientific data
* Eigen C++ template library is used for linear algebra in MLCan model
* CUSP library is used for solving sparse linear systems in GPU for integrated flow modeling

### 2.2 Workflow
Figure 3 presents the workflow of Dhara model in a GPU computing node on supercomputers. 
Typically, GPU computing nodes have a GPU device and a number of CPUs.
When computing in parallel, a number of processes run the model at the same time. The first MPI process, which we call the *master* or *root* process, also controls data movement between processes and communication with the GPU device.

<img src="http://hydrocomplexity.net/images/dhara/dhara_blocks.png?cache=none" alt="" width="700">

</br>
<left>*__Figure 3. Workflow of Dhara model in hybrid CPU-GPU parallel computing architecture__ - MPI is activated at the beginning of the program for parallel computing in MLCan model. Only the master process (rank=0) initializes CUDA environment for integrated flow modeling. Other processes can see the device (GPU) but don't need to work on it. Simulations of MLCan model on all processes are gathered/scattered to/from the master process for communicating with device for integrated flow modeling. After simulations are completed, MPI and CUDA environments are terminated.*</left>

As shown in the figure, a number of input data and parameters is required to run the model. The list of these inputs for `Dhara` are shown below:

1. **Multi-layer canopy model**
    * *Vegetation* (each plant species)
        * Canopy: Leaf Area Index - `LAI`, Leaf Area Density - `LAD`
        * Root: root density - `rootfr`
        * Parameters: Plant specie parameters are also required for running MLCan module. A list of important parameters for MLCan module can be found in its original document [here](http://onlinelibrary.wiley.com/doi/10.1029/2010JG001340/full).
        
    * *Forcings*
        * Precipiation
        * Air temperature
        * Incoming shortwave radiation
        * Longwave radiation (optional)
        * Vapor pressure
        * Atmospheric pressure
        * Wind velocity
        * Sun zenith angle
        
2. **Integrated flow model**
    * *Overland flow*
        * Topographic data
        * Initial and boundary conditions
        * Parameters: Manning's coefficients
    * *Subsurface flow*
        * Soil parameters
        * Initial and boundary conditions

## 3. Getting started

### 3.1 Bioenergy crop example
The study site in this example is located at the energy farm managed by [Energy Biosciences Institute (EBI)](http://www.energybiosciencesinstitute.org) at the [University of Illinois](https://illinois.edu). 
We will refer to the site as EBI from now on. 
Figure 4 shows a satellite image of EBI site taken in 2008.

<img src="http://hydrocomplexity.net/images/dhara/ebi_map.png?cache=none" alt="" width="800">

</br>
<center>*__Figure 4. Satellite image of EBI study site__ - The sites is 400m x 400m and has 4 subplots that plant several energy crops including corn (2 plots, top), miscanthus (1 plot, bottom left), and switchgrass (1 plot, bottom right).*</center>

It is important to make sure that all inputs are valid and reasonable before running Dhara model. 
First, let's take a look and visualize all input data at this site using jupyter notebook. 
Inputs are located at: 

<pre>
czo-mi$ Dhara_folder/examples/1-EBI
</pre>    

Similar to the batch environment in Unix, we can set Dhara folder as a variable named `$Dhara` in this jupyter notebook for fast changing directory in the future.

<pre>
Dhara='~/czomi/users/<span style="color:#cc7a00"><b>YOUR_USERNAME</b></span>/Dhara'
</pre>

Change directory to Dhara_folder:
<pre>
cd $Dhara
</pre>

Now, list the content in Dhara folder:
<pre>
ls -l
</pre>

Then, list the content in the EBI site example to screen:
<pre>
ls -l examples/1-EBI/
</pre>


<sub> __Note__: these above commands must be executed in separate jupyter python cells to be effective.</sub>

__TODO:__ create new cells and execute above commands

We see several __configuration__ (`.cfg`) and __netcdf__ (`.nc`) files in the folder. 
They all contain information needed to drive the model.
While configuration files includes *constant parameters* and *model options*, netcdf files often includes *temporal* and *spatial* datasets$^{\ddagger}$.
There is one <span style="color:#cc7a00"> __portable batch system__</span> (`.pbs`) file for submitting jobs on supercomputer that will be explained later.
We'll use [Python](https://www.python.org) to visualize and analyze all the inputs and model outputs.

<sup>$^{\ddagger}$All temporal and spatial inputs/outputs in `Dhara` are stored in `NetCDF4` format.</sup>

#### 3.1.1 Understanding model inputs
##### Import python modules
The first thing we'll do is to import all python modules/packages we need for loading and analyzing input data. Modules are generally imported at the very beginning of each notebook.

In [ ]:
# Numerical packages
import numpy as np

# NetCDF data format
from netCDF4 import Dataset

# Plotting package
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
from ipywidgets import *

# Seaborn: useful for graphics & statistics
import seaborn as sns
sns.set_style(style='white')

# Bokeh: package for interactive plotting
import bokeh.io
import bokeh.mpl
import bokeh.plotting

# functions to make matplotlib and bokeh inline;
%matplotlib inline
bokeh.io.output_notebook()

##### Configuration files
We will examine configuration files that are used to navigate the model. Go back to your jupyter main tab, navigate to `$Dhara/examples/1-EBI/` folder and open `ebi.cfg` file.

__*TO DO:*__ Open all other configurations file shown in `ebi.cfg` file. 


##### Forcing data
Now we know which `netcdf` files will be loaded into Dhara and for what reasons. Specifically, they are `forcings_7536.nc`, `topography.nc`, and 3 vegetation files (`corn.nc`, `miscanthus.nc`, and `switchgrass.nc`).

Next, let's examine forcing data which is one of the main drivers of the model. This data is stored in `forcings_7536.nc` as mentioned above.
The `plot_1Dinput()` function written below can be used to plot forcing data in jupyter notebook.

In [ ]:
"""
Function for loading and plotting 1D forcing input
"""
def plot_1Dinput(file_name, var_name):
    nc_fid = Dataset(file_name, 'r')
    x = nc_fid.variables['DecimalDayOfYear'][:] # common variable on x-axis
    var = nc_fid.variables[var_name][:]
    
    # Plotting data
    fig, ax = plt.subplots(figsize=(9,5))
    ax.plot(x,var)
    ax.set_xlabel('Time (day)')
    ax.set_ylabel(var_name)
    
    # make the plot interactive with Bokeh package
    bokeh.plotting.show(bokeh.mpl.to_bokeh())

Now, we want to pass the variable names to an interactive function, so it will allow us to change the variable we want to plot without running the code again.

In [ ]:
varname = 'AirTemperature'
filename = './examples/1-EBI/forcings_7536.nc' 

In [ ]:
# Plot data in regular mode
plot_1Dinput()

In [ ]:
# Plot data in interactive mode
# used fixed() if you do not want to change the input
interact()

Actually, there are a few more variables in the input forcing file. <br/>
__TODO__: Add the following to `varname` and re-plot the forcing data:

    * GlobalRadiation
    * LongwaveDownward
    * Precipitation
    * VaporPressureAir
    * WindVelocity
    * ZenithAngle
    * AtmosphericPressure

##### Vegetation
Vegetation netcdf files contain information about Leaf Area Index (`LAI`), Leaf Area Density (`LAD`), and root fraction (`rootfr`). Let's now take a look at these files for the different types of vegetation.

###### Leaf Area Index
The first piece of information is LAI. For better comparison, we will plot LAI for corn, miscanthus and switchgrass together.

In [ ]:
"""
Function for loading and plotting LAI data
"""
def plot_LAI(filename):
    num_plants = len(filename)
    fig, ax = plt.subplots(figsize=(9,5))        
    for i in range(0,num_plants):
        nc_fid = Dataset(filename[i], 'r')
        LAI = nc_fid.variables['LeafAreaIndex'][:]
        ax.plot(LAI, label=filename[i])

    ax.set_xlabel('Time')
    ax.set_ylabel('LAI')
    #ax.legend()
    bokeh.plotting.show(bokeh.mpl.to_bokeh())

In [ ]:
cropnames = ['./examples/1-EBI/corn.nc',
             './examples/1-EBI/miscanthus.nc',
             './examples/1-EBI/switchgrass.nc']

# TODO -- complete the arguments
plot_LAI(cropnames)

###### Leaf Area Density and Root fraction

Leaf Area Density and root fraction are also important vegetation parameters. This time, we will plot these two parameters side by side.

In [1]:
"""
Function for loading and plotting LAI data
"""
def plot_LAD_Root(filename):
    num_plants = len(filename)
    fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(10,5))        
    for i in range(0,num_plants):
        nc_fid = Dataset(filename[i], 'r')
        LAD = nc_fid.variables['LeafAreaDensity'][:]
        nl_can = nc_fid.variables['NumCanoyLayers'][:]
        ylad = np.linspace(1,nl_can,nl_can)
        rootfr = nc_fid.variables['RootFraction'][:]
        nl_soil = nc_fid.variables['NumSoilLayers'][:]
        ysoil = np.linspace(1,nl_soil,nl_soil)
        
        ax[0].plot(LAD, ylad/nl_can, label=filename[i])
        ax[1].plot(rootfr, -ysoil/nl_soil, label=filename[i])

    ax[0].set_xlabel('Leaf Area Density')
    ax[0].set_ylabel('z/h [-]')
    ax[0].grid(True)

    ax[1].set_xlabel('Root fraction')
    ax[1].set_ylabel('z/d [-]')
    ax[1].grid(True)
    ax[1].legend(loc=4)

In [ ]:
# TODO -- complete the arguments
plot_LAD_Root(cropnames)

##### Topography
We next look at topographic data of the EBI site that will be used for integrated flow model. 
As we know, topography file of EBI named `topographyEBI.nc` is also in `netCDF` format. 
The variable inside this netCDF file that stores information on topography is named as `Topography`.

In [ ]:
def plot_topography(filetopo):
    # Load topography in netCDF format
    nc_fid = Dataset(filetopo, 'r')
    topo = nc_fid.variables['Topography'][:] 

    # Set font size and light source
    font_size = 14
    ls = LightSource(azdeg=315, altdeg=45)

    # Create a figure with 2 subplots
    fig,ax = plt.subplots(ncols=2, nrows=1, figsize=(12,8))


    # SUBPLOT 0 ---------
    im0 = ax[0].imshow(topo, cmap=plt.cm.Spectral_r) # ... Plot topography
    ct = ax[0].contour(topo,
                       np.arange(np.floor(topo.min()),np.ceil(topo.max()),1.0),
                       linewidths=0.5,
                       cmap=plt.cm.jet_r) # ... Plot contour
    fig.colorbar(im0, ax=ax[0], pad = 0.1, orientation='horizontal')
    ax[0].clabel(ct, inline=True, fmt='%1.1f', fontsize=10) 


    # SUBPLOT 1 ---------
    im1 = ax[1].imshow(ls.shade(topo, cmap=plt.cm.gray_r)) # ... Plot topography 
                                                           #     with hillshade light source
    fig.colorbar(im1,ax=ax[1], pad = 0.1, orientation='horizontal')


    # Set axis labels and font size
    for i in range(0,2):
        ax[i].set_xlabel('X [m]', size=font_size)
        ax[i].set_ylabel('Y [m]', size=font_size)
        ax[i].tick_params(axis='x', labelsize=font_size)
        ax[i].tick_params(axis='y', labelsize=font_size)

    plt.tight_layout()

In [ ]:
# TODO -- locate topography.nc file and visualize data
filetopo = ''
plot_topography(filetopo)

#### 3.1.2 Running the code for EBI
We have analyzed all configurations and inputs for EBI site. 
Next, we want to run it on ROGER which has hybrid CPU-GPU parallel computing capability.
We will request 16 CPUs & 1 GPU to run simulation as illustrated as below:

&nbsp;

<img src="http://hydrocomplexity.net/images/dhara/ebi-mpi-cuda.png?cache=none" alt="" width="500">

</br>
<left>*__Figure 5. MPI-CUDA hybrid computation in the EBI example__ - 16 MPI processes are used for MLCan model on the top and 1 GPU device is used for integrated surface-subsurface flow model. Data are exchanged to couple the two models.*</left>

Running computational jobs on the head node of supuercomputers is strictly prohibited. 
We need to request a compute node or submit a batch job on the queue for computation.
The following work must be done on `ROGER` using `terminal`, not `jupterhub`.

First, let's run an <span style="color:#cc7a00">__interactive__</span> simulation. After you login to `ROGER`, type the following to request a compute node from the login (or head) node:

<pre>
    cg-gpu01$ qsub -I -l walltime=00:30:00,nodes=1:ppn=20 -A czomi
    cg-gpuXX$ cd $Dhara/examples/1-EBI
    cg-gpuXX$ mpirun –np num_proc ../../bin/dhara -option
</pre>

Now, we submit a <span style="color:#cc7a00">__batch job__</span> to the system. This work is done by submitting a *portable batch system* file that instructs ROGER to do all the work we want. Open `ebi.pbs`:

<pre>
    cg-gpu01$ cd $Dhara/examples/1-EBI
    cg-gpu01$ nano ebi.pbs
</pre>

After you input your e-mail address and modify all other information as needed, we submit the job as follow:
<pre>
    cg-gpu01$ qsub ebi.pbs
</pre>

#### 3.1.3 Analyzing Model Results

Once your simulations are complete, we will use tools similar to those discussed above to analyze the results. Remember that all the results are saved in `netcdf` format as well.

##### 2D overland flow visualization

In [ ]:
def plot_overland(time):
    nc_f = '../examples/1-EBI/results/olf2D_'+str(time)+'.nc'
    nc_fid = Dataset(nc_f, 'r')
    water_depth = nc_fid.variables['water_depth'][:]
    water_elevation = nc_fid.variables['water_elevation'][:]    
    
    # Set font size and light source
    font_size = 14

    # Create a figure with 2 subplots
    fig,ax = plt.subplots(ncols=2, nrows=1, figsize=(12,8))

    # SUBPLOT 0 ---------
    im0 = ax[0].imshow(water_elevation, 
                       cmap=plt.cm.Spectral_r,
                       vmin=water_elevation.min(), 
                       vmax=water_elevation.max())
    fig.colorbar(im0, ax=ax[0], pad = 0.1, orientation='horizontal')
    ax[0].set_title('Water Elevation',fontsize=font_size+2)

    # SUBPLOT 1 ---------
    im1 = ax[1].imshow(water_depth, 
                       cmap=plt.cm.jet_r,
                       vmin=0.0, vmax=0.2)  
    cbar = fig.colorbar(im1, ax=ax[1], pad = 0.1, orientation='horizontal')
    ax[1].set_title('Water Depth',fontsize=font_size+2)

    # Set axis labels and font size
    for i in range(0,2):
        ax[i].set_xlabel('X [m]', size=font_size)
        ax[i].set_ylabel('Y [m]', size=font_size)
        ax[i].tick_params(axis='x', labelsize=font_size)
        ax[i].tick_params(axis='y', labelsize=font_size)

    plt.tight_layout()

In [ ]:
# TODO -- visualize overland flow using interact() with: time=(0,7500,100)
interact();

##### 3D overland flow visualization

In [ ]:
def plot_ssf(time,var,layer):
    nc_f = '../examples/1-EBI/results/ssf3D_'+str(time)+'.nc'
    nc_fid = Dataset(nc_f, 'r')
    data = nc_fid.variables[var][:]
    if (var=='moisture'):
        vmin=0.3
        vmax=0.45
        cmap=plt.cm.Spectral
    else:
        vmin=-1.0
        vmax=0.0
        cmap=plt.cm.jet_r

    fig, ax = plt.subplots(figsize=(10,8))
    im = ax.imshow(data[layer,:,:], interpolation='bilinear', 
                                    aspect='auto',
                                    vmin=vmin, vmax=vmax,
                                    cmap=cmap)
    fig.colorbar(im,ax=ax)

In [ ]:
varssf = ['moisture','pressure_head']

In [ ]:
# TODO -- visualize overland flow using interact() with: 
#    time=(0,7500,100)
#    var=varssf
#    layer=(0,14,1)
interact()

We can also visualize vertical cross sections of the soil. Make it as an exercise if you want.

##### Temporal analyses

In [ ]:
def plot_2dtime(var):
    nc_f = '../examples/1-EBI/results/stat2D.nc'
    nc_fid = Dataset(nc_f, 'r')
    data = nc_fid.variables[var][:]
    if (var=='moisture_mean'):
        vmin=0.2
        vmax=0.45
        cmap=plt.cm.Spectral
    else:
        vmin=-2.0
        vmax=0.0
        cmap=plt.cm.jet_r
        
    fig, ax = plt.subplots(figsize=(18,3))
    im = ax.imshow(data.T/len(data), 
                   interpolation='bilinear', 
                   vmax=vmax, vmin=vmin,
                   aspect='auto',
                   cmap=cmap)
    fig.colorbar(im,ax=ax)

In [ ]:
var_data = ['moisture_mean','pressure_mean']

In [ ]:
interact(plot_2dtime, var=var_data);

In [ ]:
def plot_1doutput(var_name,proc):
    nc_fid = Dataset('../examples/1-EBI/results/proc'+str(proc)+'_mlcan_1d.nc', 'r')
    var = nc_fid.variables[var_name][:]
    
    # Plotting data
    fig, ax = plt.subplots(figsize=(10,6))
    ax.plot(var)
    ax.set_ylabel(var_name)
    ax.set_xlabel('Time')    
    ax.set_title('MLCan - Proc ' + str(proc))    
    
    # Want to make it interactive with Bokeh, uncomment line below
    bokeh.plotting.show(bokeh.mpl.to_bokeh())

In [ ]:
var1Doutput = ['An_can', 'LE_can', 'H_can', 'TR_can', 'Rnrad_can']

In [ ]:
# Plot data in interactive mode
interact(plot_1doutput, var_name=var1Doutput, proc=(0,15,1))

### 3.2 Asking questions
Now, we have understood how Dhara model work. Let's use it as a tool to ask and explore some interesting scientific questions. For instance:
* What are the impacts of elevated CO<sub>2</sub> on ecohydrologic dynamics? 
* What are the impacts of air temperature increase on evapotranspiration and soil moisture dynamics?
* What are their simultaneous impacts on ecohydrology?

### 3.3 Bring your research questions

It is also important to use Dhara as a framework to study questions related to your research.